In [27]:
import json
import time
import os
import sklearn_crfsuite
import re
import nereval
import pandas as pd

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten
from sklearn.metrics import classification_report
from estnltk.taggers import Retagger
from estnltk.taggers import CompoundTokenTagger
from sklearn_crfsuite import metrics

from nervaluate import Evaluator

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR

In [28]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [29]:
class TokenSplitter( Retagger ):
    """Splits tokens into smaller tokens based on regular expression patterns.""" 
    conf_param = ['patterns', 'break_group_name']
    
    def __init__(self, patterns, break_group_name:str='end'):
        # Set input/output layers
        self.input_layers = ['tokens']
        self.output_layer = 'tokens'
        self.output_attributes = ()
        # Set other configuration parameters
        if not (isinstance(break_group_name, str) and len(break_group_name) > 0):
            raise TypeError('(!) break_group_name should be a non-empty string.')
        self.break_group_name = break_group_name
        # Assert that all patterns are regular expressions in the valid format
        if not isinstance(patterns, list):
            raise TypeError('(!) patterns should be a list of compiled regular expressions.')
        # TODO: we use an adhoc way to verify that patterns are regular expressions 
        #       because there seems to be no common way of doing it both in py35 
        #       and py36
        for pat in patterns:
            # Check for the existence of methods/attributes
            has_match   = callable(getattr(pat, "match", None))
            has_search  = callable(getattr(pat, "search", None))
            has_pattern = getattr(pat, "pattern", None) is not None
            for (k,v) in (('method match()',has_match),\
                          ('method search()',has_search),\
                          ('attribute pattern',has_pattern)):
                if v is False:
                    raise TypeError('(!) Unexpected regex pattern: {!r} is missing {}.'.format(pat, k))
            symbolic_groups = pat.groupindex
            if self.break_group_name not in symbolic_groups.keys():
                raise TypeError('(!) Pattern {!r} is missing symbolic group named {!r}.'.format(pat, self.break_group_name))
        self.patterns = patterns

    def _change_layer(self, text, layers, status):
        # Get changeble layer
        changeble_layer = layers[self.output_layer]
        # Iterate over tokens
        add_spans    = []
        remove_spans = []
        for span in changeble_layer:
            token_str = text.text[span.start:span.end]
            for pat in self.patterns:
                m = pat.search(token_str)
                if m:
                    break_group_end = m.end( self.break_group_name )
                    if break_group_end > -1 and \
                       break_group_end > 0  and \
                       break_group_end < len(token_str):
                        # Make the split
                        add_spans.append( (span.start, span.start+break_group_end) )
                        add_spans.append( (span.start+break_group_end, span.end) )
                        remove_spans.append( span )
                        # Once a token has been split, then break and move on to 
                        # the next token ...
                        break
        if add_spans:
            assert len(remove_spans) > 0
            for old_span in remove_spans:
                changeble_layer.remove_span( old_span )
            for new_span in add_spans:
                changeble_layer.add_annotation( new_span )

token_splitter = TokenSplitter(patterns=[re.compile(r'(?P<end>[A-ZÕÄÖÜ]{1}\w+)[A-ZÕÄÖÜ]{1}\w+'),\
                                         re.compile(r'(?P<end>Piebenomme)metsawaht'),\
                                         re.compile(r'(?P<end>maa)peal'),\
                                         re.compile(r'(?P<end>reppi)käest'),\
                                         re.compile(r'(?P<end>Kiidjerwelt)J'),\
                                         re.compile(r'(?P<end>Ameljanow)Persitski'),\
                                         re.compile(r'(?P<end>mõistmas)Mihkel'),\
                                         re.compile(r'(?P<end>tema)Käkk'),\
                                         re.compile(r'(?P<end>Ahjawalla)liikmed'),\
                                         re.compile(r'(?P<end>kohtumees)A'),\
                                         re.compile(r'(?P<end>Pechmann)x'),\
                                         re.compile(r'(?P<end>pölli)Anni'),\
                                         re.compile(r'(?P<end>külla)Rauba'),\
                                         re.compile(r'(?P<end>kohtowannem)Jaak'),\
                                         re.compile(r'(?P<end>rannast)Leno'),\
                                         re.compile(r'(?P<end>wallast)Kiiwita'),\
                                         re.compile(r'(?P<end>wallas)Kristjan'),\
                                         re.compile(r'(?P<end>Pedoson)rahul'),\
                                         re.compile(r'(?P<end>pere)Jaan'),\
                                         re.compile(r'(?P<end>kohtu)poolest'),\
                                         re.compile(r'(?P<end>Kurrista)kaudo'),\
                                         re.compile(r'(?P<end>mölder)Gottlieb'),\
                                         re.compile(r'(?P<end>wöörmündri)Jaan'),\
                                         re.compile(r'(?P<end>Oinas)ja'),\
                                         re.compile(r'(?P<end>ette)Leenu'),\
                                         re.compile(r'(?P<end>Tommingas)peab'),\
                                         re.compile(r'(?P<end>wäljaja)Kotlep'),\
                                         re.compile(r'(?P<end>pea)A'),\
                                         re.compile(r'(?P<end>talumees)Nikolai')])

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [30]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

In [31]:
files = {}

with open('divided_corpus.txt', 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [42]:
all_results = {}

for subdistribution in [1, 2, 3, 4, 5]:
    training_subdistributions = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            subdistribution_for_testing = y
        else:
            training_subdistributions.append(y)
    
    # Getting the filenames to be trained on from the files dictionary.
    filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}
    
    # Creating training_texts from the aforementioned filenames.
    print("Defineerin treenimistekstid.")
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open('./vallakohtufailid_json/' + str(filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                training_texts.append(json_to_text(file.read()).tag_layer(['sentences', 'morph_analysis']))
    print(f"Treenimistekstid defineeritud {time.time() - start} sekundiga.")
    # Setting up the trainer and training.
    print("\n\nAlustan nertaggeri treenimist.")
    start = time.time()
    model_dir=DEFAULT_PY3_NER_MODEL_DIR
    modelUtil = ModelStorageUtil(model_dir)
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir='test' )
    print(f"NerTagger treenitud {time.time() - start} sekundiga.")
    # Setting up the new trained nertagger and defining layers to be removed later on.
    nertagger = NerTagger(model_dir = 'test')
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    
    # Tagging the files using the new nertagger.
    print("\n\nAlustan failide taggimist.")
    start = time.time()
    for file in {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}:
        with open(find(file.replace(".json", ".txt"), "./vallakohtufailid/"), 'r', encoding='UTF-8') as f:
            text = Text(f.read())
            if f == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                text = text.replace('..', '. .')

            text = text.tag_layer(['tokens'])
            token_splitter.retag(text)
            CompoundTokenTagger(tag_initials = False, tag_abbreviations = False, tag_hyphenations = False).tag(text)
            text.tag_layer('morph_analysis')

            nertagger.tag(text)
            text.add_layer(flatten(text['ner'], 'flat_ner'))

            for x in removed_layers:
                text.pop_layer(x)
            text_to_json(text, file=os.getcwd() + "/vallakohtufailid_nertagger/" + file)
            print(f'Täägitud fail {file}')
    print(f"Failid taggitud {time.time() - start} sekundiga.")
    
    # Chaning the tags into a readable formats for the evaluator.
    print("\n\nAlustan tulemuste ammutamist.")

    #gold = []
    #test = []
    gold_ner = []
    test_ner = []

    for file in {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}:
        appendable_gold_ner = []
        appendable_test_ner = []

        if file.endswith(".json"):
            if file in files_not_working:
                continue
            else:
                with open("./vallakohtufailid_nertagger/" + str(file), 'r', encoding='UTF-8') as f_test, \
                    open("./vallakohtufailid_json_flat/" + str(file), 'r', encoding='UTF-8') as f_gold:
                        test_import = json_to_text(f_test.read())
                        gold_import = json_to_text(f_gold.read())

                        # The commented part is needed for word-level-ner.
                        '''
                        for i in range(len(gold_import['flat_gold_wordner'])):
                            tag = gold_import['flat_gold_wordner'][i].nertag[0]
                            gold.append(tag)
                        for i in range(len(test_import['flat_wordner'])):
                            tag = test_import['flat_wordner'][i].nertag[0]
                            test.append(tag)
                        '''
                        
                        for i in range(len(gold_import['gold_ner'])):
                            ner = gold_import['gold_ner'][i]
                            label = ner.nertag[0]
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_gold_ner.append({"label": label, "start": start, "end": end})

                        for i in range(len(test_import['flat_ner'])):
                            ner = test_import['flat_ner'][i]
                            label = ner.nertag[0]
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_test_ner.append({"label": label, "start": start, "end": end})

        gold_ner.append(appendable_gold_ner)
        test_ner.append(appendable_test_ner)

    evaluator = Evaluator(gold_ner, test_ner, tags=['ORG', 'PER', 'MISC', 'LOC', 'LOC_ORG'])
    results, results_per_tag = evaluator.evaluate()
    all_results[subdistribution_for_testing] = (results, results_per_tag)

Defineerin treenimistekstid.
Treenimistekstid defineeritud 245.80129384994507 sekundiga.


Alustan nertaggeri treenimist.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 347967
Seconds required: 3.729

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 34179.599218
Trial #1 (eta = 0.100000): 4280.545413
Trial #2 (eta = 0.200000): 8370.865299
Trial #3 (eta = 0.400000): 15645.778272
Trial #4 (eta = 0.800000): 35056.555895 (worse)
Trial #5 (eta = 0.050000): 3227.967920
Trial #6 (eta = 0.025000): 3023.813791
Trial #7 (eta = 0.012500): 3292.880865
Trial #8 (eta = 0.006250): 3864.657485
Trial #9 (eta = 0.003125): 4696.880808
Tria

***** Epoch #37 *****
Loss: 876.179032
Improvement ratio: 0.370136
Feature L2-norm: 86.758568
Learning rate (eta): 0.024954
Total number of feature updates: 600991
Seconds required for this iteration: 0.661

***** Epoch #38 *****
Loss: 846.719808
Improvement ratio: 0.369671
Feature L2-norm: 87.419388
Learning rate (eta): 0.024953
Total number of feature updates: 617234
Seconds required for this iteration: 0.668

***** Epoch #39 *****
Loss: 825.552916
Improvement ratio: 0.358062
Feature L2-norm: 88.062906
Learning rate (eta): 0.024951
Total number of feature updates: 633477
Seconds required for this iteration: 0.676

***** Epoch #40 *****
Loss: 805.805082
Improvement ratio: 0.342989
Feature L2-norm: 88.690588
Learning rate (eta): 0.024950
Total number of feature updates: 649720
Seconds required for this iteration: 0.676

***** Epoch #41 *****
Loss: 781.238285
Improvement ratio: 0.342561
Feature L2-norm: 89.302030
Learning rate (eta): 0.024949
Total number of feature updates: 665963
Seco

***** Epoch #78 *****
Loss: 423.951483
Improvement ratio: 0.141272
Feature L2-norm: 105.232807
Learning rate (eta): 0.024903
Total number of feature updates: 1266954
Seconds required for this iteration: 0.727

***** Epoch #79 *****
Loss: 421.382717
Improvement ratio: 0.128318
Feature L2-norm: 105.547205
Learning rate (eta): 0.024902
Total number of feature updates: 1283197
Seconds required for this iteration: 0.740

***** Epoch #80 *****
Loss: 421.337735
Improvement ratio: 0.128563
Feature L2-norm: 105.860378
Learning rate (eta): 0.024900
Total number of feature updates: 1299440
Seconds required for this iteration: 0.747

***** Epoch #81 *****
Loss: 417.056841
Improvement ratio: 0.119701
Feature L2-norm: 106.167455
Learning rate (eta): 0.024899
Total number of feature updates: 1315683
Seconds required for this iteration: 0.794

***** Epoch #82 *****
Loss: 412.592742
Improvement ratio: 0.116776
Feature L2-norm: 106.469560
Learning rate (eta): 0.024898
Total number of feature updates: 13

***** Epoch #117 *****
Loss: 301.332665
Improvement ratio: 0.091290
Feature L2-norm: 115.229092
Learning rate (eta): 0.024855
Total number of feature updates: 1900431
Seconds required for this iteration: 0.977

***** Epoch #118 *****
Loss: 310.350854
Improvement ratio: 0.035555
Feature L2-norm: 115.439314
Learning rate (eta): 0.024853
Total number of feature updates: 1916674
Seconds required for this iteration: 1.021

***** Epoch #119 *****
Loss: 304.385162
Improvement ratio: 0.062398
Feature L2-norm: 115.646478
Learning rate (eta): 0.024852
Total number of feature updates: 1932917
Seconds required for this iteration: 0.826

***** Epoch #120 *****
Loss: 300.774846
Improvement ratio: 0.053226
Feature L2-norm: 115.851673
Learning rate (eta): 0.024851
Total number of feature updates: 1949160
Seconds required for this iteration: 0.658

***** Epoch #121 *****
Loss: 298.265486
Improvement ratio: 0.093241
Feature L2-norm: 116.056481
Learning rate (eta): 0.024850
Total number of feature update

***** Epoch #159 *****
Loss: 237.074141
Improvement ratio: 0.077173
Feature L2-norm: 122.767113
Learning rate (eta): 0.024803
Total number of feature updates: 2582637
Seconds required for this iteration: 0.727

***** Epoch #160 *****
Loss: 234.482514
Improvement ratio: 0.081053
Feature L2-norm: 122.921691
Learning rate (eta): 0.024802
Total number of feature updates: 2598880
Seconds required for this iteration: 0.723

***** Epoch #161 *****
Loss: 240.911518
Improvement ratio: 0.029212
Feature L2-norm: 123.074062
Learning rate (eta): 0.024800
Total number of feature updates: 2615123
Seconds required for this iteration: 0.740

***** Epoch #162 *****
Loss: 241.055904
Improvement ratio: 0.013023
Feature L2-norm: 123.225352
Learning rate (eta): 0.024799
Total number of feature updates: 2631366
Seconds required for this iteration: 0.727

***** Epoch #163 *****
Loss: 241.120490
Improvement ratio: 0.041397
Feature L2-norm: 123.376020
Learning rate (eta): 0.024798
Total number of feature update

...

***** Epoch #57 *****
Loss: 578.808992
Improvement ratio: 0.184390
Feature L2-norm: 97.100706
Learning rate (eta): 0.024929
Total number of feature updates: 889998
Seconds required for this iteration: 0.704

***** Epoch #58 *****
Loss: 569.850625
Improvement ratio: 0.186897
Feature L2-norm: 97.530595
Learning rate (eta): 0.024928
Total number of feature updates: 905612
Seconds required for this iteration: 0.728

***** Epoch #59 *****
Loss: 552.008646
Improvement ratio: 0.204395
Feature L2-norm: 97.950311
Learning rate (eta): 0.024926
Total number of feature updates: 921226
Seconds required for this iteration: 0.799

***** Epoch #60 *****
Loss: 545.405551
Improvement ratio: 0.196173
Feature L2-norm: 98.367105
Learning rate (eta): 0.024925
Total number of feature updates: 936840
Seconds required for this iteration: 0.780

***** Epoch #61 *****
Loss: 548.577535
Improvement ratio: 0.175557
Feature L2-norm: 98.777225
Learning rate (eta): 0.024924
Total number of feature updates: 952454
Seco

***** Epoch #99 *****
Loss: 357.123972
Improvement ratio: 0.087272
Feature L2-norm: 110.741665
Learning rate (eta): 0.024877
Total number of feature updates: 1545786
Seconds required for this iteration: 0.707

***** Epoch #100 *****
Loss: 352.440404
Improvement ratio: 0.078156
Feature L2-norm: 110.989852
Learning rate (eta): 0.024876
Total number of feature updates: 1561400
Seconds required for this iteration: 0.864

***** Epoch #101 *****
Loss: 349.877969
Improvement ratio: 0.100137
Feature L2-norm: 111.234204
Learning rate (eta): 0.024874
Total number of feature updates: 1577014
Seconds required for this iteration: 0.799

***** Epoch #102 *****
Loss: 348.656517
Improvement ratio: 0.085407
Feature L2-norm: 111.476315
Learning rate (eta): 0.024873
Total number of feature updates: 1592628
Seconds required for this iteration: 0.815

***** Epoch #103 *****
Loss: 345.549925
Improvement ratio: 0.083238
Feature L2-norm: 111.715764
Learning rate (eta): 0.024872
Total number of feature updates

***** Epoch #140 *****
Loss: 272.114136
Improvement ratio: 0.040894
Feature L2-norm: 119.260687
Learning rate (eta): 0.024826
Total number of feature updates: 2185960
Seconds required for this iteration: 0.784

***** Epoch #141 *****
Loss: 269.695221
Improvement ratio: 0.082004
Feature L2-norm: 119.435616
Learning rate (eta): 0.024825
Total number of feature updates: 2201574
Seconds required for this iteration: 0.997

***** Epoch #142 *****
Loss: 270.391651
Improvement ratio: 0.029834
Feature L2-norm: 119.609194
Learning rate (eta): 0.024824
Total number of feature updates: 2217188
Seconds required for this iteration: 0.685

***** Epoch #143 *****
Loss: 259.533449
Improvement ratio: 0.064267
Feature L2-norm: 119.781303
Learning rate (eta): 0.024823
Total number of feature updates: 2232802
Seconds required for this iteration: 0.665

***** Epoch #144 *****
Loss: 260.640783
Improvement ratio: 0.080302
Feature L2-norm: 119.952811
Learning rate (eta): 0.024821
Total number of feature update

Täägitud fail L22ne_Ridala_Sinalepa_id25437_1889a.json
Täägitud fail J2rva_Tyri_Tyri-Alliku_id3082_1900a.json
Täägitud fail V6ru_R6uge_Saaluse_id10962_1879a.json
Täägitud fail V6ru_Vastseliina_Misso_id22085_1883a.json
Täägitud fail Tartu_V6nnu_Ahja_id13957_1882a.json
Täägitud fail Harju_Kose_Palvere_id16729_1881a.json
Täägitud fail Viljandi_K6pu_Suure-K6pu_id4665_1883a.json
Täägitud fail P2rnu_Tori_Tori_id25337_1890a.json
Täägitud fail Tartu_V6nnu_Ahja_id22380_1869a.json
Täägitud fail Harju_Kose_Triigi_id9421_1867a.json
Täägitud fail Tartu_Kodavere_Ranna_id15035_1862a.json
Täägitud fail Harju_Kose_Triigi_id9938_1870a.json
Täägitud fail Viru_Rakvere_S6meru_id5673_1882a.json
Täägitud fail Tartu_Otep22_Pyhaj2rve_id1540_1884a.json
Täägitud fail J2rva_Tyri_V22tsa_id22178_1912a.json
Täägitud fail Harju_Juuru_Kaiu_id9874_1882a.json
Täägitud fail Tartu_Otep22_Pyhaj2rve_id7840_1885a.json
Täägitud fail V6ru_R2pina_Kahkva_id8869_1888a.json
Täägitud fail Tartu_R6ngu_Aakre_id9106_1827a.json
Täägitu

Täägitud fail Harju_Rapla_Rapla_id18671_1869a.json
Täägitud fail Tartu_V6nnu_Ahja_id18214_1886a.json
Täägitud fail Tartu_Laiuse_Kivij2rve_id1436_1856a.json
Täägitud fail Tartu_V6nnu_Ahja_id22561_1878a.json
Täägitud fail Tartu_Kodavere_Ranna_id14286_1858a.json
Täägitud fail P2rnu_Halliste_Penuja_id758_1885a.json
Täägitud fail Saare_Kihelkonna_Atla_id6893_1872a.json
Täägitud fail Harju_Hageri_Kohila_id10769_1873a.json
Täägitud fail Tartu_V6nnu_Ahja_id16318_1883a.json
Täägitud fail J2rva_Anna_Eivere_id6239_1879a.json
Täägitud fail Tartu_Kodavere_Alatskivi_id12871_1876a.json
Täägitud fail J2rva_Tyri_V22tsa_id16931_1886a.json
Täägitud fail Harju_Harju-Madise_Padise-Kloostri_id2778_1867a.json
Täägitud fail Viljandi_K6pu_Suure-K6pu_id4977_1883a.json
Täägitud fail J2rva_Tyri_S2revere_id8673_1886a.json
Täägitud fail V6ru_R2pina_Kahkva_id14118_1889a.json
Täägitud fail L22ne_Pyhalepa_K2rdla_id22309_1867a.json
Täägitud fail L22ne_Pyhalepa_K2rdla_id25096_1877a.json
Täägitud fail Tartu_Kodavere_Alat